# **Single Copy Core Gene Replication Slope Analysis**
Author: <br />
Alyse Larkin (larkinsa@uci.edu) <br />
Assistant Project Scientist <br />
Department of Earth System Science <br />
University of California, Irvine 

## Summary
This Jupyter notebook provides the R code necessary to reproduce the analysis described in Larkin *et al.* 2022, *ISME J*, "Basin-scale biogeography of *Prochlorococcus* and SAR11 ecotype replication." Specifically, in this notebook we provide: 
- A function to calculate the terminus of replication using the median circular minimum as first described by Korem *et al.* 2015 
- A function to simultaneously fit forward and reverse linear functions to gene coverage patterns such that the slopes of the forward and reverse (i.e., "right hand" and "left hand") functions are equal 
- An example application of these functions to a marine dataset using the *Prochlorococcus* ecotype HLI 

If users wish to apply this code to their own datasets, users should first obtain: 
- A Sample x Gene matrix containing the coverage values of single cope core genes. We **strongly** recommend that coverage values be based on rarefied reads, such that the same number of reads are used for every sample in the dataset. 
- A single column matrix containing the start locations for every gene based on a reference genome 


## Functions 
Three functions are used to calculate replication slopes. The first function determines the terminus of replication using the circular minimum coverage value across all samples provided. The second function fits left hand and right coverage slopes (lhs = "left hand slope", rhs = "right hand slope") based on coverage values of single copy core genes (SCCGs) as mapped to a reference genome and is used as a starting point for the final function. The third function uses non-linear least squared regression to fit the left hand and right hand linear functions such that the slope of both are the same. 

In [ ]:
### Circular Median Minimum Terminus ### 

circularMedian_term <- function(genomeMat){
	
	genomeCov = genomeMat[, colSums(genomeMat) > 0]
	
	Pbar = c()
	for(col in 1:ncol(genomeCov)){
		rowSubMin <- genomeCov[which(genomeCov[, col] > 0), col]
		rowNam <- names(rowSubMin[which.min(rowSubMin)])
		minRow <- which(rownames(genomeCov) == rowNam)
		Pbar = c(Pbar, minRow)
	}
	
	g = nrow(genomeCov)

	tmCalc = c()
	for(t in 1:length(Pbar)){
		val = (Pbar - Pbar[t]) %% g
		tmval = max(val) - min(val)
		tmCalc = c(tmCalc, tmval)
	}
	
	tm = min(tmCalc)
	
	cirMed = (median(c((Pbar - tm) %% g)) + tm) %%  g
	
	return(round(cirMed))
}

### Fit Reverse and Forward (i.e. Left Hand and Right Hand ) Slopes ### 

## Fit Left Hand and Right Hand Linear Fit simultaneously,  different slopes 
twopartfit <- function(x, y){
	f <- function(Cx.open){
		lhs <- function(x) ifelse(x < Cx.open, Cx.open-x, 0)
		rhs <- function(x) ifelse(x < Cx.open, 0, x-Cx.open)
		fit <- lm(y ~ lhs(x) + rhs(x))
		c(summary(fit)$r.squared, summary(fit)$coef[1], summary(fit)$coef[2], summary(fit)$coef[3])
	}
	
	r2 <- function(x) -(f(x)[1])
	
	res <- optimize(r2, interval = c(min(x), max(x)))
	resout <- c(res$minimum, f(res$minimum))
	
	best_Cx <- resout[1]
	coef1 <- resout[3] #Cy
	coef2 <- resout[4] 
	coef3 <- resout[5]
	
	leftHandInt <- coef1 + best_Cx*coef2
	leftHandSl <- -coef2
	rightHandInt <- coef1 - best_Cx*coef3
	rightHandSl <- coef3 
	
	out <- c(leftHandInt, leftHandSl, rightHandInt, rightHandSl)
	print(out)
}


## Fit Left Hand and Right Hand Linear Fit simultaneously, same slopes 
lhrhNLS <- function(x, y, Cx){
	lhs <- function(Cx, A, ya) ifelse(x < Cx, A*(2*Cx - x) + ya, 0)
	rhs <- function(Cx, A, ya) ifelse(x < Cx, 0, A*x + ya)
	nlsfit <- nls(y ~ lhs(Cx, A, ya) + rhs(Cx, A, ya),  data = data.frame(x, y),  start = c(A = twopartfit(x, y)[4], ya = twopartfit(x, y)[3])) ## use differing slopes as the starting point 
	sumfit <- summary(nlsfit)
	A <- sumfit$coef[1] ## slope
	ya <- sumfit$coef[2] ## intercept
	p <- summary(nlsfit)$parameter[1, 4] ## p-value
	pe <- sqrt(mean(summary(nlsfit)$residuals^2))/(max(y) - min(y)) ## Percent Error / Range-Normalized RMSE
	R <- cor(y, predict(nlsfit))^2 ## R^2
	AIC <- AIC(nlsfit) ## AIC value 
	out <- c(A, ya, p, pe, R, AIC)
	print(out)
}

## Example Dataset
In our example dataset, we use the coverage values of 1530 single copy core genes (SCCGs) from the *Prochlorococcus* ecotype HLI using the C13.5 Bio-GO-SHIP transect (Larkin *et al.* 2021). SCCGs were identified using the Anvi'o pangenomic pipeline (Eren *et al.* 2015) and thus use "GC_", or "Gene Cluster", as a naming convention. The gene start locations were determined via the fully sequenced HLI reference genome MED4. 

In [ ]:
gene_coverages <- read.csv('HLI_GeneCoverages_C13-5.csv', row.names = 1); gene_coverages

In [ ]:
genome_order <- read.csv('HLI_GeneOrder.csv', row.names = 1); genome_order

In the code below we format the gene coverage matrix such that column order matches the gene start locations in the reference genome 

In [ ]:
## Match gene coverages to their start location in the reference genome 
gene_start_locations <- as.numeric(genome_order[, 1])

genes_ordered_match <- gene_coverages[, match(rownames(genome_order), colnames(gene_coverages))]
genes_ordered <- genes_ordered_match[order(rownames(genes_ordered_match)), ]

## Check that gene orders are matching
all(rownames(genome_order) == colnames(genes_ordered))


## Calculate Terminus 
In the code below we use all samples in our C13.5 dataset to estimate the minimum coverage location of our reference genome and designate that as the terminus of replication. We then re-order our genes based on the terminus location. 


In [ ]:
## Calculate Terminus 
terminus <- circularMedian_term(t(genes_ordered))
terminus_start <- gene_start_locations[terminus] 


## Re-order the genes based on the terminus 
new_start_locations <- c() 

for(st in 1:length(gene_start_locations)){
	m <- round(max(gene_start_locations)/2) #middle of array
	if(terminus_start < m){
		d = m - terminus_start #difference between terminus_start and m 
		t = max(gene_start_locations) - d #threshold 
		if(gene_start_locations[st] < t){
			new_start_locations <- c(new_start_locations, gene_start_locations[st] + d)
		} else if(gene_start_locations[st] >= t){
			new_start_locations <- c(new_start_locations, gene_start_locations[st] - t)
		}
		
	} else if(terminus_start > m){
		t = terminus_start - m #threshold
		d = max(gene_start_locations) - t #difference
		if(gene_start_locations[st] < t){
			new_start_locations <- c(new_start_locations, gene_start_locations[st] + d)
		} else if(gene_start_locations[st] >= t){
			new_start_locations <- c(new_start_locations, gene_start_locations[st] - t)
		}
		
	} else {
		new_start_locations <- c(new_start_locations, gene_start_locations[st])
	}
}



## Calculate Slopes
Now that our genes are ordered starting at the origin of replication, through the terminus, and back to the origin, we can calculate the replication slope using the coverage of all SCCGs across our reference genome. 
Note that if all coverage values for a particular sample are 0, then the slope is set to 0. 
If the "lhrhNLS" function does not converge to a single solution, then the slope is reported as "NA." 

In [ ]:
## Calculate the slope of coverage,  retain intercepts for plotting purposes  

slopes <- c()
intercepts <- c()

for(s in 1:nrow(genes_ordered)){
	y <- as.numeric(as.character(genes_ordered[s, ]))

	x <- as.numeric(new_start_locations)
	
	Cx <- round(max(new_start_locations)/2)
	
	if(sum(genes_ordered[s, ]) == 0){
		slopes <- c(slopes, 0)
		intercepts <- c(intercepts, 0)
	} else if(class(try(myfit <- lhrhNLS(x, y, Cx))) == "try-error") {
		slopes <- c(slopes, NA)
		intercepts <- c(intercepts, NA)
	} else {
		slopes <- c(slopes, myfit[1])
		intercepts <- c(intercepts, myfit[2])
	}
	
}

slopes_matrix <- matrix(slopes, ncol=1, nrow=length(slopes))
rownames(slopes_matrix) <- rownames(genes_ordered)

intercepts_matrix <- matrix(intercepts, ncol=1, nrow=length(intercepts))
rownames(intercepts_matrix) <- rownames(genes_ordered)


## Check Bi-linear Coverage  
In the code below we examine the first sample in our dataset to ensure that we observe a bi-linear coverage pattern. 

In [ ]:
plot(new_start_locations, genes_ordered[1, ], xlab="MED4 genome position", ylab="SCCG coverage", main=toString(rownames(genes_ordered)[1]))
abline(intercepts_matrix[1, ], slopes_matrix[1, ], col='red', lwd=1.5) #rhs
abline(2*slopes_matrix[1, ]*round(max(new_start_locations)/2) + intercepts_matrix[1, ], -slopes_matrix[1, ], col='red', lwd=1.5) #lhs

## Plot Replication 
In the code below we normalize our coverage slopes by multiplying the slope by 1/2 the length of our reference genome. This gives us our replication estimate, R<sub>Obs</sub>. We then plot the first fifty R<sub>Obs</sub> to visualize the variability in replication across the beginning of the C13.5 Bio-GO-SHIP transect. 

In [ ]:
barplot(slopes_matrix[1:50, ] * (max(new_start_locations)/2), horiz=F, beside=T, names.arg=rownames(slopes_matrix[1:50, ]), las=2, cex.names=0.3, ylab="Replication (Robs)", main="Prochlorococcus HLI")